**Connect to Elasticsearch**

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

client_info = es.info()

print("Connected to Elasticsearch!")

pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**1. Human-readable output**

In [8]:
response = es.cluster.stats(human=True)
pprint(response["nodes"]["jvm"])

{'max_uptime': '2.5h',
 'max_uptime_in_millis': 9070283,
 'mem': {'heap_max': '1gb',
         'heap_max_in_bytes': 1073741824,
         'heap_used': '284.4mb',
         'heap_used_in_bytes': 298231568},
 'threads': 89,
 'versions': [{'bundled_jdk': True,
               'count': 1,
               'using_bundled_jdk': True,
               'version': '25',
               'vm_name': 'OpenJDK 64-Bit Server VM',
               'vm_vendor': 'Oracle Corporation',
               'vm_version': '25+36-3489'}]}


**2. Date math**

In [9]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [11]:
import json 
from tqdm import tqdm 

operations = [] 

documents = json.load(open("data/data.json"))
documents[0]

{'title': 'Sample Title 1',
 'text': 'This is the first sample document text.',
 'created_on': '2024-09-22'}

In [12]:
for document in tqdm(documents, total=len(documents)):
    operations.append({"index": {"_index": "my_index"}}) # action 
    operations.append(document)

response = es.bulk(operations=operations)
pprint(response.body)

100%|██████████| 5/5 [00:00<00:00, 18859.28it/s]


{'errors': False,
 'items': [{'index': {'_id': '0WccbJoBycThPNe-O9ut',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': '0mccbJoBycThPNe-O9ut',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 1,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': '02ccbJoBycThPNe-O9ut',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 2,
                      '_shards': {'failed': 0, 'successful': 1, '

In [ ]:
query = {
    "query": {
        "range": {
            "created_on": {
                "gte": "2024-09-22||+1d/d", # 2024-09-23
                "lte": "now/d", # 2024-10-5
            }
        }
    }
}


response = es.search(
    index="my_index", 
    body=query
)

hits = response["hits"]["hits"]

for hit in hits:
    print(hit["_source"])

{'title': 'Sample Title 3', 'text': 'Django Rest Framework simplifies API development with powerful serialization tools.', 'created_on': '2024-11-14'}
{'title': 'Sample Title 4', 'text': 'Python provides a wide range of libraries for data analysis, automation, and backend development.', 'created_on': '2025-01-10'}
{'title': 'Sample Title 5', 'text': 'FastAPI is an excellent choice for building high-performance APIs with async capabilities. This is a document too', 'created_on': '2025-02-02'}


In [24]:
query = {
    "query": {
        "range": {
            "created_on": {
                "gte": "2024-09-22||+1M/d", # 2024-09-23
                "lte": "now/d", # 2024-10-5
            }
        }
    }
}


response = es.search(
    index="my_index", 
    body=query
)

hits = response["hits"]["hits"]

for hit in hits:
    print(hit["_source"])

{'title': 'Sample Title 3', 'text': 'Django Rest Framework simplifies API development with powerful serialization tools.', 'created_on': '2024-11-14'}
{'title': 'Sample Title 4', 'text': 'Python provides a wide range of libraries for data analysis, automation, and backend development.', 'created_on': '2025-01-10'}
{'title': 'Sample Title 5', 'text': 'FastAPI is an excellent choice for building high-performance APIs with async capabilities. This is a document too', 'created_on': '2025-02-02'}


**3. Response filtering**

3.1 Inclusive filtering

In [26]:
response = es.search(
    index="my_index",
    body={
        "query": {
            "match_all": {}
        }
    },
)
pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '0WccbJoBycThPNe-O9ut',
                    '_index': 'my_index',
                    '_score': 1.0,
                    '_source': {'created_on': '2024-09-22',
                                'text': 'This is the first sample document '
                                        'text.',
                                'title': 'Sample Title 1'}},
                   {'_id': '0mccbJoBycThPNe-O9ut',
                    '_index': 'my_index',
                    '_score': 1.0,
                    '_source': {'created_on': '2024-10-05',
                                'text': 'Elasticsearch makes searching and '
                                        'analyzing large amounts of data fast '
                                        'and efficient.',
                                'title': 'Sample Title 2'}},
                   {'_id': '02ccbJoBycThPNe-O9ut',
                    '_index': 'my_index'

In [28]:
response = es.search(
    index="my_index", 
    body={
        "query": {
            "match_all": {}
        }
    },
    filter_path="hits.hits._id,hits.hits._source" # Keep only _id and _source fields
)

hits = response["hits"]["hits"]

for hit in hits:
    pprint(hit)

{'_id': '0WccbJoBycThPNe-O9ut',
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Sample Title 1'}}
{'_id': '0mccbJoBycThPNe-O9ut',
 '_source': {'created_on': '2024-10-05',
             'text': 'Elasticsearch makes searching and analyzing large '
                     'amounts of data fast and efficient.',
             'title': 'Sample Title 2'}}
{'_id': '02ccbJoBycThPNe-O9ut',
 '_source': {'created_on': '2024-11-14',
             'text': 'Django Rest Framework simplifies API development with '
                     'powerful serialization tools.',
             'title': 'Sample Title 3'}}
{'_id': '1GccbJoBycThPNe-O9ut',
 '_source': {'created_on': '2025-01-10',
             'text': 'Python provides a wide range of libraries for data '
                     'analysis, automation, and backend development.',
             'title': 'Sample Title 4'}}
{'_id': '1WccbJoBycThPNe-O9ut',
 '_source': {'created_on': '2025-02-

3.2 Exclusive filtering

In [29]:
response = es.search(
    index="my_index", 
    body={
        "query": {
            "match_all": {}
        }
    },
    filter_path="-hits"
)

pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'timed_out': False,
 'took': 2}


3.3. Combined filtering

In [30]:
response = es.search(
    index="my_index", 
    body={
        "query": {
            "match_all": {}
        }
    },
    filter_path="hits.hits._id,-hits.hits._source"
)

pprint(response.body)

{'hits': {'hits': [{'_id': '0WccbJoBycThPNe-O9ut'},
                   {'_id': '0mccbJoBycThPNe-O9ut'},
                   {'_id': '02ccbJoBycThPNe-O9ut'},
                   {'_id': '1GccbJoBycThPNe-O9ut'},
                   {'_id': '1WccbJoBycThPNe-O9ut'}]}}


**4. Flat settings**

In [31]:
response = es.indices.get_settings(
    index="my_index"
)

pprint(response.body)

{'my_index': {'settings': {'index': {'creation_date': '1762750189342',
                                     'number_of_replicas': '1',
                                     'number_of_shards': '1',
                                     'provided_name': 'my_index',
                                     'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
                                     'uuid': 'veSvIn4KTWWZNW5rkQvJjw',
                                     'version': {'created': '9039001'}}}}}


In [32]:
response = es.indices.get_settings(
    index="my_index",
    flat_settings=True
)

pprint(response.body)

{'my_index': {'settings': {'index.creation_date': '1762750189342',
                           'index.number_of_replicas': '1',
                           'index.number_of_shards': '1',
                           'index.provided_name': 'my_index',
                           'index.routing.allocation.include._tier_preference': 'data_content',
                           'index.uuid': 'veSvIn4KTWWZNW5rkQvJjw',
                           'index.version.created': '9039001'}}}
